In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df = pd.read_csv('Train.csv')

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pickle

# Initialize the VADER sentiment analyzer
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

# Assuming df is your DataFrame containing text data and Combined_clean is the column
# containing the text you want to analyze
# Replace 'Combined_clean' with the actual column name in your DataFrame
sentiment_labels_list = []
for text in df['Combined_clean']:
    sentiment_scores = sid.polarity_scores(text)
    # Extract the compound score
    compound_score = sentiment_scores['compound']
    # Assign labels based on the compound score
    if compound_score > 0.05:  # Positive sentiment
        sentiment_label = "happy"
    elif compound_score < -0.05:  # Negative sentiment
        sentiment_label = "complaint"
    else:  # Neutral sentiment
        sentiment_label = "default"
    # Append the text and its sentiment label to the list
    sentiment_labels_list.append((text, sentiment_label))

# Save sentiment labels to a .pkl file
with open('sentiment_labels.pkl', 'wb') as f:
    pickle.dump(sentiment_labels_list, f)

print("Sentiment labels saved successfully.")


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Sentiment labels saved successfully.


In [ ]:
# prompt: calculate unique values from predicted_sentiment

unique_sentiments = df['predicted_sentiment'].unique()
print(f"Unique sentiment categories: {unique_sentiments}")


In [ ]:
# prompt: iwant to calculate value count of each predicted_sentimednt column

sentiment_count = df['predicted_sentiment'].value_counts()
print(f"Sentiment Value Counts:\n{sentiment_count}")


Sentiment Value Counts:
predicted_sentiment
enquiry         1279
satisfied        380
default          208
dissatisfied      82
emergency         23
Name: count, dtype: int64


In [ ]:
df.head()

,industry,Combined_clean,industry_label,predicted_sentiment,Lenght_before,Lenght_after,predicted_sentiment_label
0,Pharmaceutical,amount included less year reflects anticipated...,2,enquiry,2542,2542,3
1,Energy,see note financial statements description mone...,0,default,2876,2876,0
2,Finance,rwas federal reserve board riskbased capital r...,1,enquiry,3197,3197,3
3,Technology,commitments contingencies time time ordinary c...,3,satisfied,1862,1862,4
4,Finance,pnc financial services group inc form determin...,1,enquiry,3457,3457,3


In [ ]:
import numpy as np
import pandas as pd
!pip install beautifulsoup4
import os
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm # Progress Bar
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import transformers
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import warnings
from transformers import logging as hf_logging
hf_logging.set_verbosity_error() # Hidding Huggingface Warnings
warnings.filterwarnings("ignore")
import torch

In [ ]:
MODEL_NAME = 'distilbert-base-cased'

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)



unique_labels = df_train['predicted_sentiment'].unique()
print(unique_labels)

industry_counts = df_train['predicted_sentiment'].value_counts()

print(industry_counts)

df_train.head()

le = LabelEncoder()
df_train['predicted_sentiment_label'] = le.fit_transform(df_train['predicted_sentiment'])

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df_train['Combined_clean'].values, df_train['predicted_sentiment_label'].values, test_size=0.1, random_state=42)

df_train.head()

label_dict = (df_train[['predicted_sentiment','predicted_sentiment_label']].drop_duplicates()
              .sort_values(by='predicted_sentiment_label')
              .reset_index(drop=True)['predicted_sentiment']
              .to_dict())

for index, key in label_dict.items():
    print(index, key)

from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

!pip install accelerate>=0.21.0
!pip install transformers>=4.11.3
!pip install transformers[torch]

# Install the required libraries (if not already installed)
# !pip install transformers

import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

# Load your preprocessed dataset (replace with your actual data)
# Assuming you have 'X_train', 'y_train', 'X_val', and 'y_val'

# Load the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Encode the text data
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True)

# Create PyTorch datasets
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings["input_ids"]),
    torch.tensor(train_encodings["attention_mask"]),
    torch.tensor(y_train),
)
val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings["input_ids"]),
    torch.tensor(val_encodings["attention_mask"]),
    torch.tensor(y_val),
)

# Load the DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=5)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./distilbert_model",  # Directory to save the model
    per_device_train_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=500,
    logging_dir="./logs",
    num_train_epochs=1
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=lambda data: {"input_ids": torch.stack([f[0] for f in data]),
                                "attention_mask": torch.stack([f[1] for f in data]),
                                "labels": torch.stack([f[2] for f in data])},
)

# Train the model
trainer.train()

# Save the trained model
model.save_pretrained("path/to/save/directory")

import pickle

# Evaluate the model on the validation dataset
eval_results = trainer.evaluate()

# Get the predicted labels for the validation dataset
y_pred = trainer.predict(val_dataset).predictions.argmax(axis=1)

# Get the true labels for the validation dataset
y_true = val_dataset[:][2].numpy()

# Calculate the accuracy
accuracy = accuracy_score(y_true, y_pred)

# Print the accuracy
print("Accuracy:", accuracy)

# Load the PyTorch model
model_path = "path/to/save/directory"  # Replace with the path where your model is saved
model = DistilBertForSequenceClassification.from_pretrained(model_path)

# Save the model as a .pkl file using pickle
with open("distilbert_model.pkl", "wb") as f:
    pickle.dump(model, f)

text ='''Dear manager,
I recently had a payment issue while using your company. The reason I was given was "Due to energy issue, the payment servers were down for a while, we are sorry for the issue, we will ensure this won't happen again." The reason was not ok for me, I feel really unsatisfied and energy was not good. If I was travelling some where and I faced this issue, what should I do? If there is a pharmaceutical emergency?? This kind of technological issues should not be happening with such a brand, you have trust of billions of customers, thank you.
'''

# Preprocess text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)

# Get predicted class
predicted_class = torch.argmax(outputs.logits).item()


['enquiry' 'default' 'satisfied' 'emergency' 'dissatisfied']
predicted_sentiment
enquiry         1279
satisfied        380
default          208
dissatisfied      82
emergency         23
Name: count, dtype: int64
0 default
1 dissatisfied
2 emergency
3 enquiry
4 satisfied


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

{'eval_loss': 0.6887059211730957, 'eval_runtime': 184.7538, 'eval_samples_per_second': 1.072, 'eval_steps_per_second': 0.135, 'epoch': 0.45}
{'eval_loss': 0.5965902805328369, 'eval_runtime': 179.0771, 'eval_samples_per_second': 1.106, 'eval_steps_per_second': 0.14, 'epoch': 0.9}
{'train_runtime': 6490.8929, 'train_samples_per_second': 0.273, 'train_steps_per_second': 0.034, 'train_loss': 0.7759560937280053, 'epoch': 1.0}
{'eval_loss': 0.5802134275436401, 'eval_runtime': 182.352, 'eval_samples_per_second': 1.086, 'eval_steps_per_second': 0.137, 'epoch': 1.0}
Accuracy: 0.8080808080808081


In [ ]:
# Map predicted class to label
labels = ["default", "dissatisfied", "emergency","enquiry","satisfied"]  # Replace with your actual class labels
predicted_label = labels[predicted_class]

print("Predicted Label:", predicted_label)

label_dict = {0: 'default', 1: 'dissatisfied', 2: 'emergency', 3: 'enquiry', 4: 'satisfied'}

def predict_category(text):


    inputs = tokenizer(cleaned_text, return_tensors="pt", truncation=True, padding=True)

    outputs = model(**inputs)

    predicted_label = torch.argmax(outputs.logits).item()

    predicted_category = label_dict[predicted_label]

    return predicted_category


text_input = input("Enter the text: ")
predicted_category = predict_category(text_input)
print("Predicted Category:", predicted_category)



Predicted Label: enquiry
